In [22]:
using RDST, Distributions, CSV, DataFrames, ProgressMeter

In [23]:
const mrgParams = MRG32k3a([12, 3, 45, 67, 8, 9])
const mrgGumbel = copy(mrgParams)
next_substream!(mrgGumbel)

Full state of MRG32k3a generator:
Cg = [3362321159, 3940116905, 1930530409, 4096561893, 4003684126, 4194805046]
Bg = [3362321159, 3940116905, 1930530409, 4096561893, 4003684126, 4194805046]
Ig = [12, 3, 45, 67, 8, 9]

In [24]:
const nchoice = 5
const nInd = 1_000_000

1000000

In [25]:
function genFibbomod(N::Int)
    if N == 1
        return [0]
    elseif N == 2
        return [0, 1]
    else
        rt = [0, 1]
        for k in 3:N
            push!(rt, (rt[k - 2] + rt[k - 1]) % 100)
        end
        return rt
    end
end

genFibbomod (generic function with 1 method)

In [27]:
function genLogitInd(N::Int)
    params = [quantile.(Normal(), rand(mrgParams, N)) for _ in 1:nchoice]
    
    beta = genFibbomod(N)
    vals = [beta'*p for p in params] + quantile.(Gumbel(), rand(mrgGumbel, nchoice))
    sortp = sortperm(-vals)
    paramsasVec = vcat(params[sortp]...)
    return paramsasVec
end

genLogitInd (generic function with 1 method)

In [28]:
function genLogitPop(N::Int)
    df = DataFrame(zeros(0, N*nchoice), :auto)
    reset_substream!(mrgParams)
    reset_substream!(mrgGumbel)
    for _ in 1:nInd
        push!(df, genLogitInd(N::Int))
    end
    return df
end

genLogitPop (generic function with 2 methods)

In [29]:
function genLogitPop(N::Int, nchoice::Int, nInd::Int)
    df = DataFrame(zeros(0, N*nchoice), :auto)
    reset_substream!(mrgParams)
    reset_substream!(mrgGumbel)
    for _ in 1:nInd
        push!(df, genLogitInd(N::Int))
    end
    return df
end

genLogitPop (generic function with 2 methods)

In [30]:
df = genLogitPop(5)
CSV.write("L5.csv", df)

df = genLogitPop(10)
CSV.write("L10.csv", df)

df = genLogitPop(50)
CSV.write("L50.csv", df)

df = genLogitPop(100)
CSV.write("L100.csv", df)

"L100.csv"